In [13]:
import numpy as np
from sklearn.decomposition import PCA

In [84]:
# data
n = 100
p = 10
q = 5 # dims to keep
X = np.random.randn(n,p)
eye = np.ones(n)

We have a data matrix $X \sim n \times p$. Its (Bessel-corrected) covariance matrix can be written $C = \frac{1}{n-1} X^T X - \frac{1}{n^2} \mu \mu^T$ where $\mu$ is the p-vector of column means. We demean $X$ first, so the covariance matrix is just the first term above.

In [85]:
# First demean. The vector of column means is obtained simply by dotting with an n-vector of ones on the left, and dividing by n.
mu = eye@X / n
X_dm = X-mu


In [86]:
# covariance matrix
cov = np.cov(X_dm,rowvar=False)
cov2= X_dm.T@X_dm/(n-1)
np.testing.assert_almost_equal(cov,cov2)

Now we perform a standard eigendecomposition. We decompose $C$ as

$C = \frac{1}{n-1} Q D Q^T$

where $Q$ is the matrix whose columns are the eigenvectors of $C$, and $D$ is the diagonal matrix of e-values.

In [87]:
# eigendecomposition of covariance matrix
(evalues, Q) = np.linalg.eig(cov)

# sort order of e-values & e-vectors
idx = np.argsort(evalues)[::-1]
evalues = evalues[idx]
Q = Q[:,idx]

# confirm that the decomposition works
check = Q @ np.diag(evalues) @ Q.T
np.testing.assert_almost_equal(check,cov)


With $Q$ in hand we can transform our data. We define

$Z = XQ$

and observe that the covariance matrix of $Z$ is diagonal:

$C_Z = \frac{1}{n-1} D$

In [94]:
# define new variables Z. They are linear combinations of the old variables.
Z = X_dm@Q # both Z and X_dm are nxp


Now keep only the first $q$ variables. These are the ones that explain the most variance.

In [95]:
# The fraction of variance explained by each is given by the sum of remaining e-values
frac = evalues/np.sum(evalues)
frac

array([0.1507622 , 0.13428279, 0.12666082, 0.10918671, 0.09557138,
       0.08983471, 0.08153292, 0.07807001, 0.07574734, 0.05835113])

In [96]:
# define Z-t (truncated) which keeps only the first q dimensions
Z_t = Z[:,0:q]


Another way to do PCA is by using the singular value decomposition on the data matrix (not on the covariance matrix).

In [98]:
(U, s, VT) = np.linalg.svd(X_dm)

d = np.diag(s)
sigma = np.pad(d,((0,n-p), (0,0)), 'constant')
np.testing.assert_almost_equal(X_dm, U@sigma@VT)

# define new variables Z
Z = X_dm@VT.T

#sigma.T@sigma/(n-1)

In [99]:
# Now do it using sklearn
pca = PCA(q)
pca.fit(X)
Z = pca.transform(X)
pca.explained_variance_ratio_

array([0.1507622 , 0.13428279, 0.12666082, 0.10918671, 0.09557138])